<a href="https://colab.research.google.com/github/navtej16/Emotion_Recgonition_using_audio_and_video/blob/main/Emotion_recognition_using_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import tensorflow as tf

import keras
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D , AveragePooling2D
from keras.layers import Dense , Activation , Dropout ,Flatten

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator



In [2]:
#from google.colab import files
#uploaded = files.upload()

In [5]:
from google.colab import drive
drive.mount("/content/gdrive")
import pandas as pd
data = pd.read_csv('/content/gdrive/My Drive/fer2013.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:
#data.shape
#data.head()

x_test = []
y_test = []
x_train = []
y_train = []

import numpy as np

l = np.array(data);

for index,row in data.iterrows():
  k = row['pixels'].split(" ")

  emot = row['emotion']
  emot = keras.utils.to_categorical(emot,7)

  if row['Usage'] == 'Training' :
    x_train.append(np.array(k))
    y_train.append(emot)
  elif row['Usage'] == 'PublicTest' :
    x_test.append(np.array(k))
    y_test.append(emot)
  
x_train = np.array(x_train,'float32')
y_train = np.array(y_train,'float32')
x_test = np.array(x_test,'float32')
y_test = np.array(y_test,'float32')

#print(x_train.shape)
#print(y_train)
#print(x_test)

x_train /=255
x_test /=255

x_train = x_train.reshape(x_train.shape[0],48,48,1)
x_test = x_test.reshape(x_test.shape[0],48,48,1)
#print(x_train.shape)






(28709, 2304)
(28709, 48, 48, 1)


In [30]:
model = Sequential()

#1st 1ayer
model.add(Conv2D(64,(5,5),activation='relu',input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5),strides=(2,2)))

#2nd 1ayer
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

#3rd 1ayer
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))

model.add(Flatten())

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(7,activation='softmax'))

gen = ImageDataGenerator()
train_gen = gen.flow(x_train,y_train,batch_size=256)

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

fit = True 

if fit == True:
   model.fit_generator(train_gen,steps_per_epoch = 256 , epochs = 25)
else:
  model.load_weights('/data/facial_expression_model_weights.h5')








/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
256/256 [==============================] - 202s 780ms/step - loss: 1.8248 - accuracy: 0.2506
